## Solving QAP with SA Algorithm

##### AhmadReza Nopoush                    610301194

at first, we import the data we need.

In [1]:
import random
import math

at second, we input the data by reading the file. at the end we have two matrix. "D" which keeps the distance of two places or "Adjacency matrix" if you prefer. and "F" which defines the flow (weight, number of connections) between pairs of facilities.

In [33]:
#D is a matrix that keeps the distance of two places or "Adjacency matrix" if you prefer.
D = list()

#F defines the flow (weight, number of connections) between pairs of facilities.
F = list()

flag = 0

#note that you can input your file by changing the address below.
with open("esc32a.txt",mode="r") as line:
    for l in line.readlines():
        if flag==0:
            O = int(l)
            flag = 1
            List = list()
        
        #inputing D    
        elif flag == 1:
            L = [int(i) for i in l.split()]
            List += L
            if len(List) == O:
                D.append(List)
                List = list()
            if len(D) == O:
                flag = 2
                List = list()
        
        #inputing F        
        elif flag == 2:
            L = [int(i) for i in l.split()]
            List += L
            if len(List) == O:
                F.append(List)
                List = list()
            if len(F) == O:
                flag = 3
                List = list()      


third, We define cost function due to formula $$ \phi(\pi) = \sum_{i=1}^{n} \sum_{j=1}^{n} d_{ij} f_{\pi_i \pi_j} $$


In [3]:
#Cost function: return Cost of a permutation.
def Cost(Permutation:list,D:list,F:list) -> int:
    n = len(Permutation)
    sum = 0
    for i in range(n):
        for j in range(n):
            sum += (D[i][j] * F[Permutation[i]][Permutation[j]])
    return sum

fourth, we define neighborhood for our solutions. if the solution is in the shape of permutation, then we can generate a neighbor by swaping random two index value of the permutation. we implement it as below:

In [4]:
#return a neighbor to a permutation by swapping 2 random indexes.
def neighbour(size:int, Permutation:list) -> list:
    neighbor = Permutation[:]
    
    # 2 random indexes
    i = random.randint(0,size-2)
    j = random.randint(i+1,size-1)
    
    #swap
    neighbor[i],neighbor[j] = neighbor[j],neighbor[i]
    
    return neighbor

fifth, we define our main algorithms as "Simulated_Annealing" function. this function inputs the variables:
1. S_init: initial solution, usually random shuffle of numbers.
2. StrTemp: initial amount of temprature.
3. coolfact: the cooling rate, a rate that reduce the temprature in each iteration.
4. FrzTemp: freez temprature, the final and minimum acceptable value of the temprature.

in the function, we define three kinds of solutions:
1. S_curr: the main solution in the algorithm, the neighbors are genereated from this solution.
2. S_best: the best solution found so far.
3. S_new: a new neighbor solution which generated.

then we define the "flag" varibale. when we stuck in local optima, we should increase the temrature so we can accept a random solution with higher probability. this helps the power of exploration in our algorithm. when we don't find a solution which is not as good as S_best or S_curr, we increament the flag. when flag reachs to 100, we increase temprature so more random solution will be generated and we can go out from local optima.

at the end, we return S_best. you can read more about algorithm in coments



In [5]:
#main Algorithm
def Simulated_Annealing(S_init, StrTemp, coolfact, FrzTemp):
    #initializing the starting variable of SA.
    Temp = StrTemp
    
    #S_cur is copied from S_init
    S_curr = S_init.copy()   
    Scurr_cost = Cost(S_curr,D,F)
    
    #S_best initializing
    S_best = S_curr.copy()
    Sbest_cost = Scurr_cost
    
    flag  = 1
    
    #repeat until Temp is in bound.
    while Temp > FrzTemp:
        
        #increasing Temp
        if flag == 100:
            Temp *= (2-coolfact)
        
        #genearte a neighbor for S_curr
        S_new = neighbour(len(S_init),S_curr)
        Snew_cost = Cost(S_new, D, F)
        
        #delta
        delta_Cost = Snew_cost - Scurr_cost
        
        #if our S_new is better than S_best, replace S_best with S_new.
        if Snew_cost < Sbest_cost:
            S_best = S_new.copy()
            Sbest_cost = Snew_cost
            
            #reset flag
            flag = 1
        
        #if S_new is better than S_curr, replace S_curr with S_new
        if delta_Cost < 0:
            S_curr = S_new.copy()
            Scurr_cost = Snew_cost
        
        #replace S_curr with probability
        elif random.random() < math.exp(-delta_Cost / Temp):
            S_curr = S_new.copy()
            Scurr_cost = Snew_cost
        
        #nothing happened, so increase the flag! 
        else:
            flag += 1
        
        #decrease Temprature
        Temp *= coolfact
        
        print(Scurr_cost, Temp)
    
    return S_best, Sbest_cost

Now, we test our algorithm:

In [37]:
qs = list(range(O))
random.shuffle(qs)
Simulated_Annealing(qs,1000000,0.999,0.00001)

406 999000.0
402 998001.0
402 997002.999
432 996005.9960009999
432 995009.990004999
432 994014.9800149939
430 993020.9650349789
406 992027.944069944
374 991035.916125874
376 990044.8802097482
348 989054.8353295384
358 988065.7804942089
358 987077.7147137147
358 986090.6369990009
398 985104.5463620019
390 984119.44181564
404 983135.3223738243
398 982152.1870514505
442 981170.034864399
444 980188.8648295345
452 979208.6759647051
444 978229.4672887403
420 977251.2378214516
428 976273.9865836302
444 975297.7125970466
444 974322.4148844496
444 973348.0924695651
448 972374.7443770955
438 971402.3696327184
478 970430.9672630857
486 969460.5362958226
492 968491.0757595268
492 967522.5846837673
498 966555.0620990836
482 965588.5070369844
454 964622.9185299474
454 963658.2956114175
450 962694.6373158061
434 961731.9426784903
436 960770.2107358119
436 959809.440525076
440 958849.6310845509
422 957890.7814534663
434 956932.8906720128
436 955975.9577813408
476 955019.9818235595
476 954064.961841736

([20,
  22,
  18,
  2,
  6,
  4,
  0,
  27,
  3,
  16,
  29,
  13,
  21,
  23,
  19,
  11,
  31,
  17,
  25,
  7,
  8,
  28,
  10,
  24,
  30,
  15,
  26,
  12,
  9,
  5,
  1,
  14],
 140)

### Results

#### chr12a
by the parameters:
1. strTemp = 10000
2. coolfact = 0.9999
3. FrzTemp = 100

we got to:

1. ([6, 4, 11, 1, 0, 2, 8, 10, 9, 5, 7, 3], 9552)
2. ([6, 4, 11, 1, 0, 2, 8, 10, 9, 5, 7, 3], 9552)
3. ([6, 4, 11, 1, 0, 2, 8, 10, 9, 5, 7, 3], 9552)

#### esc32a
by the parameters:
1. strTemp = 1000000
2. coolfact = 0.999
3. FrzTemp = 0.00001

we got to:

1. ([24, 9, 28, 20, 25, 17, 22, 6, 13, 30, 27, 18, 16, 1, 21, 2, 23, 29, 31, 5, 11, 19, 10, 14, 7, 3, 15, 12, 4, 0, 26, 8], 158)
2. ([17, 3, 7, 5, 9, 29, 10, 22, 1, 15, 28, 23, 25, 11, 13, 18, 31, 4, 12, 27, 6, 21, 2, 14, 20, 19, 30, 24, 0, 8, 16, 26], 154)
3. ([2, 9, 17, 21, 1, 5, 29, 30, 25, 7, 4, 14, 0, 8, 24, 26, 6, 16, 20, 13, 19, 12, 18, 23, 28, 10, 22, 31, 3, 15, 27, 11], 154)

#### nug20
by the parameters:
1. strTemp = 100
2. coolfact = 0.99999
3. FrzTemp = 1

we got to:

1. ([5, 0, 6, 4, 16, 12, 7, 19, 14, 18, 15, 10, 11, 1, 3, 8, 2, 9, 13, 17], 2570)
2. ([16, 7, 10, 3, 15, 4, 19, 14, 1, 18, 12, 6, 11, 17, 13, 5, 0, 9, 2, 8], 2594)
3. ([8, 15, 13, 1, 17, 2, 9, 11, 14, 18, 12, 10, 7, 19, 3, 5, 0, 4, 6, 16], 2578)
4. ([16, 6, 4, 0, 5, 3, 19, 7, 10, 12, 18, 14, 11, 9, 15, 17, 1, 13, 2, 8], 2574)

#### tai30a
by the parameters:
1. strTemp = 1000000
2. coolfact = 0.9999
3. FrzTemp = 0.01

we got to:

1. ([21, 15, 2, 8, 4, 5, 10, 18, 27, 24, 0, 7, 26, 28, 14, 23, 16, 11, 22, 20, 17, 12, 3, 19, 6, 1, 13, 25, 29, 9], 1888344)
2. ([17, 14, 24, 7, 21, 25, 13, 3, 4, 23, 0, 16, 8, 6, 20, 2, 12, 9, 26, 19, 11, 29, 5, 18, 10, 15, 22, 28, 27, 1], 1888252)
3. ([7, 22, 1, 24, 23, 8, 18, 14, 15, 3, 19, 17, 11, 12, 16, 4, 0, 21, 27, 25, 29, 10, 9, 6, 26, 20, 2, 28, 5, 13], 1886356)

#### lipa50a
by the parameters:
1. strTemp = 100000000
2. coolfact = 0.999
3. FrzTemp = 0.001

we got to:

1. ([39, 31, 26, 10, 36, 9, 18, 12, 38, 4, 41, 20, 15, 48, 37, 33, 44, 32, 22, 8, 6, 43, 19, 16, 46, 49, 34, 24, 42, 13, 5, 45, 11, 23, 7, 3, 2, 21, 14, 17, 30, 0, 1, 28, 47, 25, 40, 29, 27, 35], 62902)
2. ([7, 18, 17, 19, 49, 1, 34, 14, 33, 30, 22, 16, 36, 35, 32, 11, 28, 47, 12, 31, 26, 3, 13, 6, 41, 25, 0, 27, 2, 24, 44, 9, 15, 4, 21, 38, 8, 10, 39, 45, 40, 48, 46, 29, 42, 23, 43, 5, 37, 20], 62888)
3. ([38, 1, 37, 18, 6, 27, 30, 2, 3, 23, 41, 31, 47, 28, 40, 13, 25, 33, 46, 14, 22, 12, 29, 49, 48, 36, 35, 5, 21, 9, 10, 4, 34, 42, 20, 0, 32, 15, 43, 26, 44, 19, 7, 16, 24, 8, 45, 17, 11, 39] 62829)




